In [6]:
import pandas as pd
import requests
import datetime
import yfinance as yf
import time

In [11]:
def buscar_venda_call(ativo, data_vencimento, distancia_strike=1):

    cookies = {
    '_gid': 'GA1.3.721123878.1766674907',
    '.AspNetCore.Antiforgery.g1ir8UL7PVw': 'CfDJ8PB93EeIUJdKtD-b-s3_nBbWvt39IFdasNe93aOAl6wmMHrxy3_zxDF0MB_PemjuSvMAQwGQddnfzQkAHjYNEixzpVZqgKT7d_LEuF57IqBQO6OWxG1bigOJk_elz5D-qawLrIPBuB8WUEPgyXdzmyo',
    '_ga': 'GA1.3.916223412.1764780575',
    '_ga_YH2ELJFQPC': 'GS2.3.s1766763605$o10$g0$t1766763605$j60$l0$h0',
    '.AspNetCore.Identity.Application': 'CfDJ8PB93EeIUJdKtD-b-s3_nBYa2c8jYKnRrJpllz852UEmP8_sEXjoynHU7c4dE8J9eDQVNLKSgya8AhvAMsnqkTua9FyJd4-2UeWXnKnd6AjQzmUQmEMQgnZBylzG2R0NMho3VPDvGzCZdCrXRhm-0XYNLs9eJlTqPrkKBFKWMRlHgavXCFcEbJ7pUfo4P963iCBvTtEVzaJkg5JlAJWXnRmAYSxIjjZzvfvMOs8xgFHEPzEvtBRlccp9bVYlDaN2eJ63M-sTkEPPubK6oBD7X8NuTXLpJxVfXnw15JOdYqNF-1RCqo1NoM9T4PCgCEBzwHI_EaNd3UlZ-YW02LOb0wfl8XFha9F8fS02VW8YEa5mQuCs-kFDSQp7jQLb07ZnbMGyuviI4CdEJXRqywyeb-JLHX-QjfZjAKcEjCkWX_P7lN8hI8mTiMsHC-Cir6KXdV3wCngyfysxmALeHLp2DruX8Jru23uLtFClOJG6M9yhlc9O6E-eiaM06Ar4Npt7L0TGSt47MUf36_7T9WVIIjl77CHGSvIQzEI_sge1JmkWTa4caNUkvITTTGa7zwrrzMS84fw_8aYaRxJeYoMKrxAX0bA0Evkjz7HztaUw-6Q3GRsSq9CkiJZvw_earfaCbE2bCHC6RPk7-L24Y3_lRAuW812Bd09nQrmJtJSqJydI3nzCJsCOhbeuSkXNwalPUw',
    }
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
    }
    
    params = {'idAcao': ativo, 'dataReferencia': datetime.datetime.now().strftime('%Y-%m-%d')}
    
    url = 'https://opcoes.net.br/listaopcoes/todas'
    response = requests.get(url, params=params, cookies=cookies, headers=headers)
    dados_json = response.json()

    puts = dados_json['optionsChain']['CALL']

    linhas = []
    
    for vencimento, info_venc in puts.items():
        du = info_venc.get('du')
        strikes = info_venc.get('strikes', {})
    
        for strike, valores in strikes.items():
            linhas.append({
                'vencimento': vencimento,
                'du': du,
                'strike': float(strike.replace(',', '.')),
                'codigo': valores[0],
                'flags': valores[1],
                'data_negocio': valores[2],
                'premio': (
                    float(valores[3].replace(',', '.'))
                    if valores[3] not in (None, '')
                    else None
                ),
                'iv': (
                    float(valores[4].replace(',', '.'))
                    if valores[4] not in (None, '')
                    else None
                )
            })

    ticker = yf.Ticker(f"{ativo}.SA")
    preco_atual = ticker.info.get('bid') or ticker.info.get('currentPrice')


    df_put = pd.DataFrame(linhas)[['codigo','strike','premio','vencimento','data_negocio']].dropna()

    df_put['ativo'] = ativo.upper()
    
    df_put['cotacao'] = preco_atual
    
    df_put['(strike + premio)'] = df_put['strike'] + df_put['premio']
    
    df_put['rentabilidade'] = round((df_put['premio']/df_put['strike']) * 100,3)
    
    df_put['vencimento'] = pd.to_datetime(df_put['vencimento'])
    
    df_put['data_negocio'] = pd.to_datetime(df_put['data_negocio'].map(lambda x: x.split(" ")[0]), dayfirst=True)
    
    df_put['distancia'] = round(abs(df_put['strike']/df_put['cotacao']),3)
    
    df_put['intriseco'] = abs(df_put['strike'] - df_put['cotacao'])
    
    df_put['extrinseco'] = abs(df_put['premio'] - df_put['intriseco'])
    
    df_put = df_put[['ativo','codigo','cotacao','strike','distancia','(strike + premio)','rentabilidade','premio','vencimento','data_negocio']]

    distancia_strike = distancia_strike
    alvo = preco_atual * distancia_strike
    
    df_put = df_put[
        
        (df_put['distancia'] <= distancia_strike) &
        ((df_put['vencimento'] == data_vencimento)) &
        ((df_put['data_negocio'] == df_put['data_negocio'].max()))
        
    ].sort_values('rentabilidade',ascending=False).head(10)

    return df_put

In [12]:
def execVendaDePutOpcoes(data_vencimento='2026-02-20'):
    
    ativos = ['ABEV3', 'B3SA3', 'BBAS3', 'BBSE3', 'EGIE3', 'FLRY3', 'HYPE3', 'ITSA4', 'KLBN11',
              'LEVE3', 'PETR4', 'TAEE11', 'UNIP6', 'VALE3', 'WEGE3', 'AGRO3', 'TUPY3', 'WIZC3']
    
    distancias_alvo = [1,1.1,1.2]
    
    # Dicionário para armazenar os resultados de cada distância
    resultados_por_distancia = {d: [] for d in distancias_alvo}

    for ativo in ativos:
        try:
            print(f"Buscando cadeia completa de {ativo}...")
            
            # 1. Fazemos APENAS UMA requisição para o ativo
            # Passamos distancia_strike=999 (ou um valor alto) para garantir que pegamos tudo
            df_completo_ativo = buscar_venda_call(ativo, data_vencimento, distancia_strike=10)
            
            if df_completo_ativo.empty:
                continue

            # 2. Agora filtramos o DF baixado para cada distância da nossa lista
            for d in distancias_alvo:
                # Filtra os dados que já estão na memória
                # (Supondo que a lógica de 'distancia' já venha calculada da sua função buscar_venda_put)
                temp_df = df_completo_ativo[df_completo_ativo['distancia'] >= d].head(3).copy()
                
                if not temp_df.empty:
                    temp_df['distancia_alvo'] = d
                    resultados_por_distancia[d].append(temp_df)
            
            # Pequeno delay apenas entre ativos diferentes
            time.sleep(1)

        except Exception as e:
            print(f"Erro ao processar {ativo}: {e}")

    # 3. Agora percorremos o dicionário para printar os DataFrames por distância
    lista_final = []
    for d in distancias_alvo:
        if resultados_por_distancia[d]:
            df_dist = pd.concat(resultados_por_distancia[d], ignore_index=True)
            
            lista_final.append(df_dist)
    
    return pd.concat(lista_final, ignore_index=True) if lista_final else pd.DataFrame()

In [13]:
df = execVendaDePutOpcoes(data_vencimento='2026-03-20')

Buscando cadeia completa de ABEV3...
Buscando cadeia completa de B3SA3...
Buscando cadeia completa de BBAS3...
Buscando cadeia completa de BBSE3...
Buscando cadeia completa de EGIE3...
Buscando cadeia completa de FLRY3...
Buscando cadeia completa de HYPE3...
Buscando cadeia completa de ITSA4...
Buscando cadeia completa de KLBN11...
Buscando cadeia completa de LEVE3...
Buscando cadeia completa de PETR4...
Buscando cadeia completa de TAEE11...
Buscando cadeia completa de UNIP6...
Buscando cadeia completa de VALE3...
Buscando cadeia completa de WEGE3...
Buscando cadeia completa de AGRO3...
Buscando cadeia completa de TUPY3...
Buscando cadeia completa de WIZC3...


In [14]:
valor_investido = 5000
df["cotas"] = (valor_investido/df["strike"] // 100 *100)
df["cotas x premio"] = df["premio"] * df["cotas"]
df.sort_values('rentabilidade', ascending=False)[['ativo', 'codigo', 'cotacao', 'strike', 'distancia',
       '(strike + premio)', 'rentabilidade', 'premio', 'vencimento',
       'data_negocio', 'cotas', 'cotas x premio']]



,ativo,codigo,cotacao,strike,distancia,(strike + premio),rentabilidade,premio,vencimento,data_negocio,cotas,cotas x premio
2,B3SA3,C138,13.33,13.50,1.013,14.42,6.815,0.92,2026-03-20,2026-01-02,300.0,276.0
22,WEGE3,C500,48.10,48.36,1.005,51.36,6.203,3.00,2026-03-20,2026-01-02,100.0,300.0
19,PETR4,C315,30.63,30.65,1.001,32.46,5.905,1.81,2026-03-20,2026-01-02,100.0,181.0
3,B3SA3,C141,13.33,13.75,1.032,14.54,5.745,0.79,2026-03-20,2026-01-02,300.0,237.0
23,WEGE3,C501,48.10,48.86,1.016,51.66,5.731,2.80,2026-03-20,2026-01-02,100.0,280.0
20,PETR4,C322,30.63,31.40,1.025,33.15,5.573,1.75,2026-03-20,2026-01-02,100.0,175.0
5,BBAS3,C223,21.60,22.22,1.029,23.44,5.491,1.22,2026-03-20,2026-01-02,200.0,244.0
21,PETR4,C317,30.63,30.90,1.009,32.53,5.275,1.63,2026-03-20,2026-01-02,100.0,163.0
6,BBAS3,C220,21.60,21.97,1.017,23.11,5.189,1.14,2026-03-20,2026-01-02,200.0,228.0
4,B3SA3,C145,13.33,14.00,1.050,14.66,4.714,0.66,2026-03-20,2026-01-02,300.0,198.0


In [16]:
df.sort_values('rentabilidade', ascending=False)[['ativo', 'codigo', 'cotacao', 'strike', 'distancia',
       '(strike + premio)', 'rentabilidade', 'premio', 'vencimento',
       'data_negocio', 'cotas', 'cotas x premio']].to_excel('opcoesCALL.xlsx')